# Домашнее задание 4: Рекомендательные системы и Spark MLlib

ФИО: Алёхин Арсений Михайлович

Группа: ИУ6-54Б

## Определение варианта по фамилии

In [1]:
surname = "Алёхин" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  1
Задача № 2:  1


## Инициализация Spark и загрузка данных

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

spark = SparkSession.builder.appName("HW4").master("local[*]").getOrCreate()

# ВЫБЕРИТЕ ОДИН ИЗ ДВУХ ПУТЕЙ:
path = "ml-latest-small/"     # Малый датасет (100K рейтингов)
# path = "ml-latest/"             # Полный датасет (30M рейтингов)

movies = spark.read.csv(path + "movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv(path + "ratings.csv", header=True, inferSchema=True)

movies.cache()
ratings.cache()

print(f"Фильмов загружено: {movies.count()}")
print(f"Рейтингов загружено: {ratings.count()}")

target_genres = ["Animation", "Romance", "Documentary"]

Фильмов загружено: 9742
Рейтингов загружено: 100836


26/01/11 15:23:38 WARN CacheManager: Asked to cache already cached data.
26/01/11 15:23:38 WARN CacheManager: Asked to cache already cached data.


### ЗАДАНИЕ 1: Анализ датасета


Вариант 1. Animation, Romance, Documentary

⚠️ Замечание: Один фильм может принадлежать разным жанрам

1. Выведите данные, сопоставляющие жанры и количество фильмов
2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом

In [ ]:
df_g = movies.withColumn("genre", F.explode(F.split("genres", "\\|")))

stats = ratings.groupBy("movieId").agg(
    F.count("rating").alias("cnt"),
    F.avg("rating").alias("avg_r")
)

joined = df_g.join(stats, "movieId")
joined.cache()

print("\n1. КОЛИЧЕСТВО ФИЛЬМОВ ПО ЖАНРАМ:")
print("-" * 80)
df_g.filter(F.col("genre").isin(target_genres)).groupBy("genre").count().orderBy("genre").show()

for genre in target_genres:
    print(f"\n{'='*80}")
    print(f"ЖАНР: {genre}")
    print(f"{'='*80}")
    
    data = joined.filter(F.col("genre") == genre)
    
    print(f"\n2. ТОП 10 ПО КОЛИЧЕСТВУ ОЦЕНОК:")
    data.orderBy(F.col("cnt").desc()).select("title", "cnt").limit(10).show(10, False)
    
    print(f"\n3. ТОП 10 С НАИМЕНЬШИМ КОЛИЧЕСТВОМ:")
    data.filter("cnt > 10").orderBy(F.col("cnt").asc()).select("title", "cnt").limit(10).show(10, False)
    
    print(f"\n4. ТОП 10 ПО МАКСИМАЛЬНОМУ СРЕДНЕМУ РЕЙТИНГУ:")
    data.filter("cnt > 10").orderBy(F.col("avg_r").desc()).select("title", "avg_r").limit(10).show(10, False)
    
    print(f"\n5. ТОП 10 ПО МИНИМАЛЬНОМУ СРЕДНЕМУ РЕЙТИНГУ:")
    data.filter("cnt > 10").orderBy(F.col("avg_r").asc()).select("title", "avg_r").limit(10).show(10, False)

26/01/11 15:23:41 WARN CacheManager: Asked to cache already cached data.



1. КОЛИЧЕСТВО ФИЛЬМОВ ПО ЖАНРАМ:
--------------------------------------------------------------------------------
+-----------+-----+
|      genre|count|
+-----------+-----+
|  Animation|  611|
|Documentary|  440|
|    Romance| 1596|
+-----------+-----+


ЖАНР: Animation

2. ТОП 10 ПО КОЛИЧЕСТВУ ОЦЕНОК:
+---------------------------+---+
|title                      |cnt|
+---------------------------+---+
|Toy Story (1995)           |215|
|Aladdin (1992)             |183|
|Lion King, The (1994)      |172|
|Shrek (2001)               |170|
|Beauty and the Beast (1991)|146|
|Finding Nemo (2003)        |141|
|Monsters, Inc. (2001)      |132|
|Incredibles, The (2004)    |125|
|Up (2009)                  |105|
|WALL·E (2008)              |104|
+---------------------------+---+


3. ТОП 10 С НАИМЕНЬШИМ КОЛИЧЕСТВОМ (>10):
+--------------------------------------+---+
|title                                 |cnt|
+--------------------------------------+---+
|Happy Feet (2006)                     

### ЗАДАНИЕ 2: Коллаборативная фильтрация

Вариант 1. По схожести пользователей

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
3. Определите rmse для тестового подмножества


In [8]:
train_init, test = ratings.randomSplit([0.8, 0.2], seed=42)
train_init.cache()
test.cache()

print(f"\nОбучающая выборка: {train_init.count()}")
print(f"Тестовая выборка: {test.count()}")

gl_avg = train_init.select(F.avg("rating")).first()[0]
print(f"Среднее значение: {gl_avg:.4f}")

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

print("БАЗОВАЯ МОДЕЛЬ (всегда среднее):")
test_base = test.withColumn("prediction", F.lit(gl_avg))
rmse_base = evaluator.evaluate(test_base)
print(f"RMSE: {rmse_base:.10f}")

print("USER-BASED COLLABORATIVE FILTERING:")
user_avgs = train_init.groupBy("userId").agg(F.avg("rating").alias("user_avg"))
item_avgs = train_init.groupBy("movieId").agg(F.avg("rating").alias("item_avg"))
preds = test.join(user_avgs, "userId", "left").join(item_avgs, "movieId", "left")
preds = preds.withColumn(
    "prediction",
    F.when(
        (F.col("user_avg").isNotNull()) & (F.col("item_avg").isNotNull()),
        (F.col("user_avg") + F.col("item_avg")) / 2
    ).when(F.col("user_avg").isNotNull(), F.col("user_avg"))
    .when(F.col("item_avg").isNotNull(), F.col("item_avg"))
    .otherwise(F.lit(gl_avg))
)

rmse_cf = evaluator.evaluate(preds)
print(f"RMSE: {rmse_cf:.10f}")
print(f"Улучшение: {(rmse_base - rmse_cf) / rmse_base * 100:+.2f}%")

26/01/11 15:28:06 WARN CacheManager: Asked to cache already cached data.
26/01/11 15:28:06 WARN CacheManager: Asked to cache already cached data.



Обучающая выборка: 80578
Тестовая выборка: 20258
Среднее значение: 3.5039
БАЗОВАЯ МОДЕЛЬ (всегда среднее):
RMSE: 1.0504377707
USER-BASED COLLABORATIVE FILTERING:
RMSE: 0.8986371318
Улучшение: +14.45%


### ЗАДАНИЕ 3: Факторизация матрицы (ALS)

1. Выберите модель ALS по минимальному значению rmse. Для этого используйте кросс-валидацию k-folds c k=4

Параметры:

Количество факторов: [5, 10, 15]
Регуляризация: [0.001, 0.01, 0.1, 1, 10]
⚠️ Замечание: Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то rmse будет NaN

2. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов

In [9]:
print("\nПараметры: rank=[5,10,15], regParam=[0.001,0.01,0.1,1,10]")
print("Cross-Validation: 4-fold")

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", 
          coldStartStrategy="drop", seed=42)

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 15]) \
    .addGrid(als.regParam, [0.001, 0.01, 0.1, 1, 10]) \
    .build()

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, 
                    evaluator=evaluator, numFolds=4, seed=42)

model = cv.fit(train_init)
best_model = model.bestModel

print(f"\nЛучший rank: {best_model.rank}")
print(f"Лучший regParam: {best_model._java_obj.parent().getRegParam()}")

res_als = best_model.transform(test)
rmse_als = evaluator.evaluate(res_als)
print(f"RMSE: {rmse_als:.10f}")


Параметры: rank=[5,10,15], regParam=[0.001,0.01,0.1,1,10]
Cross-Validation: 4-fold


26/01/11 15:28:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/01/11 15:28:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK



Лучший rank: 5
Лучший regParam: 0.1
RMSE: 0.8755777027


## Сравнение результатов

In [11]:
print(f"\nБазовая модель:         RMSE = {rmse_base:.10f}")
print(f"Коллаборативная фильтр: RMSE = {rmse_cf:.10f}")
print(f"Факторизация матрицы:   RMSE = {rmse_als:.10f}")

print(f"\nУлучшения:")
print(f"CF:  {(rmse_base - rmse_cf) / rmse_base * 100:+.2f}%")
print(f"ALS: {(rmse_base - rmse_als) / rmse_base * 100:+.2f}%")



Базовая модель:         RMSE = 1.0504377707
Коллаборативная фильтр: RMSE = 0.8986371318
Факторизация матрицы:   RMSE = 0.8755777027

Улучшения:
CF:  +14.45%
ALS: +16.65%
